# <span style="color: rgb(138, 92, 245);">Segmentos de Jugadores con Características Similares en FIFA 19</span>

---------------------

Los conjuntos de datos proporcionados incluyen los atributos detallados de cada jugador
registrado en la última edición de la base de datos FIFA 19 (videojuego FIFA 2019). Atributos como el sueldo,
la posición, nacionalidad, etc. Se cuenta con más de 18K registros y 89 atributos. El objetivo del dataset es encontrar segmentos de jugadores, los cuales comparten características.

Este notebook puede ser encontrado en el repositorio de [github.com/AuriRaven/Unsupervised_Learning](https://github.com/AuriRaven/Unsupervised_Learning).

Se comienza por leer el dataset y revisar sus características básicas. 

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/data_fifa.csv')

In [3]:
df.head()

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18207 entries, 0 to 18206
Data columns (total 88 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        18207 non-null  int64  
 1   Name                      18207 non-null  object 
 2   Age                       18207 non-null  int64  
 3   Photo                     18207 non-null  object 
 4   Nationality               18207 non-null  object 
 5   Flag                      18207 non-null  object 
 6   Overall                   18207 non-null  int64  
 7   Potential                 18207 non-null  int64  
 8   Club                      17966 non-null  object 
 9   Club Logo                 18207 non-null  object 
 10  Value                     18207 non-null  object 
 11  Wage                      18207 non-null  object 
 12  Special                   18207 non-null  int64  
 13  Preferred Foot            18159 non-null  object 
 14  Intern

# <span style="color: rgb(138, 92, 245);">I. Clasificación de Variables</span>

Coniderando este resumen del dataset, se continua por identificar los tipos de variables. De esta manera será más fácil la ingeniería y limpieza que las variables puedan necesitar 

**Tipos de variables:**
* Variables numericas : Discretas y continuas
* Variables categoricas
* Variables tipo fecha
* Variables de texto : comentarios, descripciones, url
* Variables geograficas

## <span style="color: rgb(138, 92, 245);">1. Variables numéricas</span>

### 🔷 Numéricas discretas (enteros, conteos, categorías codificadas en números)

* `ID`

* `Age`

* `Overall`

* `Potential`

* `Special`

* `Jersey Number` (aunque viene como float, conceptualmente es discreta)

### 🔷 Numéricas continuas (tienen decimales, medidas, ratings)

(Nota: muchas son “skills” de posiciones y vienen como float)

#### **Posiciones (rating por posición):**

* `LS`

* `ST`

* `RS`

* `LW`

* `LF`

* `CF`

* `RF`

* `RW`

* `LAM`

* `CAM`

* `RAM`

* `LM`

* `LCM`

* `CM`

* `RCM`

* `RM`

* `LWB`

* `LDM`

* `CDM`

* `RDM`

* `RWB`

* `LB`

* `LCB`

* `CB`

* `RCB`

* `RB`

#### **Atributos técnicos:**

* `International Reputation`

* `Weak Foot`

* `Skill Moves`

* `Crossing`

* `Finishing`

* `HeadingAccuracy`

* `ShortPassing`

* `Volleys`

* `Dribbling`

* `Curve`

* `FKAccuracy`

* `LongPassing`

* `BallControl`

#### **Atributos físicos y de movimiento:**

* `Acceleration`

* `SprintSpeed`

* `Agility`

* `Reactions`

* `Balance`

* `ShotPower`

* `Jumping`

* `Stamina`

* `Strength`

* `LongShots`

* `Height` (formato “5'11”) Se neceista limpieza y convertir a numérica

* `Weight` (formato “159lbs”) Se neceista limpieza y convertir a numérica

#### **Atributos económicos y contractuales**

* `Value` (formato texto: "€110M") Se neceista limpieza y convertir a numérica

* `Wage` (formato texto: "€200K") Se neceista limpieza y convertir a numérica

* `Release Clause` (formato texto "€120M") Se neceista limpieza y convertir a numérica

#### **Atributos mentales y defensivos:**

* `Aggression`

* `Interceptions`

* `Positioning`

* `Vision`

* `Penalties`

* `Composure`

* `Marking`

* `StandingTackle`

* `SlidingTackle`

#### **Atributos de portero (GK):**

* `GKDiving`

* `GKHandling`

* `GKKicking`

* `GKPositioning`

* `GKReflexes`

In [5]:
# Funcion para convertir variables monetarias a un numero flotante
def money_to_float(x):
    if isinstance(x, str):
        x = x.replace("€", "")
        if "M" in x:
            return float(x.replace("M", "")) * 1_000_000
        if "K" in x:
            return float(x.replace("K", "")) * 1_000
    return None

# Función para convertir altura de pies y pulgadas a cm
def height_to_cm(h):
    if isinstance(h, str):
        feet, inches = h.split("'")
        return int(feet) * 30.48 + int(inches) * 2.54
    return None

# Función para convertir peso de libras a kg
def weight_to_kg(w):
    if isinstance(w, str):
        return float(w.replace("lbs","")) * 0.453592
    return None

In [6]:
df["Value"] = df["Value"].apply(money_to_float)

df["Wage"] = df["Wage"].apply(money_to_float)

df["Release Clause"] = df["Release Clause"].apply(money_to_float)

In [7]:
df["Height"] = df["Height"].apply(height_to_cm)

In [8]:
df["Weight"] = df["Weight"].apply(weight_to_kg)

## <span style="color: rgb(138, 92, 245);">2. Variables Categóricas</span>

### 🔶 Categóricas nominales (no tienen orden)

* `Name`

* `Nationality`

* `Club`

* `Preferred Foot`

* `Work Rate`

* `Body Type`

* `Position`

* `Loaned From`

### 🔶 Categóricas binarias (Sí/No, True/False)

* `Real Face` (Sí / No)
(También considerada como nominal)


## <span style="color: rgb(138, 92, 245);">3. Variables tipo fecha</span>

(Son objetos de texto, pero representan fechas y deben convertirse)

* `Joined`

* `Contract Valid Until`

(Después deben transformarse con pd.to_datetime.)

In [9]:
# Función para convertir variables de tipo fecha
def convert_date_columns(dataframe, date_columns):
    for column in date_columns:
        dataframe[column] = pd.to_datetime(dataframe[column], errors='coerce')
    return dataframe[date_columns].dtypes

In [10]:
columnas_fecha = ['Joined', 'Contract Valid Until']

convert_date_columns(df, columnas_fecha)

Joined                  datetime64[ns]
Contract Valid Until    datetime64[ns]
dtype: object

## <span style="color: rgb(138, 92, 245);">4. Variables de texto</span>

(URLs, rutas, descripciones, strings sin estructura)

* `Photo`

* `Flag`

* `Club Logo`

Se optará por eliminar estas variables pues no describen habilidad, rendimiento ni atributos del jugador.

In [11]:
df.drop(columns=['Photo','Flag','Club Logo'], inplace=True)

## <span style="color: rgb(138, 92, 245);">5. Variables geográficas</span>

* `Nationality` (origen geográfico, aunque categórica, pertenece también a esta categoría)

Si más adelante tienes columnas como ciudad o país del club, también van aquí.

# <span style="color: rgb(138, 92, 245);">II. Limpieza e Ingeniería de Datos</span>

## <span style="color: rgb(138, 92, 245);">1. Completitud</span>

In [12]:
def check_missing_values(dataframe):
    missing_values = dataframe.isnull().sum()
    missing_percentage = (missing_values / len(dataframe)) * 100
    missing_df = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage})
    return missing_df

In [13]:
check_missing_values(df).head(44)

,Missing Values,Percentage
ID,0,0.000000
Name,0,0.000000
Age,0,0.000000
Nationality,0,0.000000
Overall,0,0.000000
Potential,0,0.000000
Club,241,1.323667
Value,252,1.384083
Wage,241,1.323667
Special,0,0.000000


In [14]:
check_missing_values(df).tail(44)

,Missing Values,Percentage
LDM,2085,11.451639
CDM,2085,11.451639
RDM,2085,11.451639
RWB,2085,11.451639
LB,2085,11.451639
LCB,2085,11.451639
CB,2085,11.451639
RCB,2085,11.451639
RB,2085,11.451639
Crossing,48,0.263635


In [15]:
# Funcion para eliminar columnas con mas del 20% de valores nulos
def drop_high_na_columns(df, threshold=0.2):
    na_fraction = df.isna().mean()
    cols_to_drop = na_fraction[na_fraction >= threshold].index
    df.drop(columns=cols_to_drop, inplace=True) 
    print(f"Columnas eliminadas: {list(cols_to_drop)}")

In [16]:
drop_high_na_columns(df, threshold=0.2)

Columnas eliminadas: ['Loaned From']


## <span style="color: rgb(138, 92, 245);">2. Valores Extremos</span>

## <span style="color: rgb(138, 92, 245);">3. Duplicidad</span>

## <span style="color: rgb(138, 92, 245);">4. EDA</span>

# <span style="color: rgb(138, 92, 245);">IV. Ingeniería de Variables</span>

# <span style="color: rgb(138, 92, 245);">V. Normalización</span>

# <span style="color: rgb(138, 92, 245);">Reducción de Variables</span>

# <span style="color: rgb(138, 92, 245);">VI. Selección de Número de Grupos</span>

# <span style="color: rgb(138, 92, 245);">VII. Clustering</span>

# <span style="color: rgb(138, 92, 245);">VIII. Visualización de Grupos</span>

# <span style="color: rgb(138, 92, 245);">IX. Perfilamiento</span>